# Análise das Stories

Notebook para o auxílio da análise das stories do chatbot.

### Configurando e Imports

In [9]:
import rasa_nlu
import rasa_core
from IPython.display import IFrame

print("Versões utilizadas neste jupyter notebook:")
print("rasa_nlu: {} rasa_core: {}".format(rasa_nlu.__version__, rasa_core.__version__))


Versões utilizadas neste jupyter notebook:
rasa_nlu: 0.15.0 rasa_core: 0.14.1


## Análise e Avaliação das Stories

### Visualizaçõa do Fluxo de Conversa

O comando abaixo monta um grafo com a relação das `intents` e `utters` do chatbot, ou seja, você vai conseguir visualizar o fluxo de conversa do seu chatbot.

Esta visualização é importante para verificar possíveis problemas na estrutura do seu chatbot e se ele realmente chega em determinadas "**falas**" da conversa.

In [10]:
!python -m rasa_core.visualize -d $COACH_DOMAIN_PATH -s $COACH_STORIES_PATH -o chat_graph.html

/usr/local/lib/python3.6/runpy.py:125: RuntimeWarning: 'rasa_core.visualize' found in sys.modules after import of package 'rasa_core', but prior to execution of 'rasa_core.visualize'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecat

O Rasa gerou uma página `html` com o grafo de conversa, para facilitar, você pode visualizar o grafo no arquivo `chat_graph.html` aqui na próxima celula.

Use o mouse para dar zoom e arrastar sobre o conteúdo do grafo. Altere `width` e `height` se desejar.

* Dica: caso você se perca com **zoom in** ou **zoom out** no grafo, apeas re-execute a célula abaixo e ele irá reaparecer na célula.

In [11]:
IFrame(src='./chat_graph.html', width=900, height=700)

### Avaliação das Stories

Outra forma de analisar seu chatbot é por meio da própria avaliação do Rasa, ele gera uma matriz de confuzão com os dados fornecidos nas `stories` e do resultado do treinamento armazenado na pasta `models`.

* Caso você ainda não tenha treinado seu chatbot execute a céclula abaixo para treina-lo.

In [ ]:
!make train -C $COACH_DIR_PATH

make: Entering directory '/work/coach'
python3 -m rasa_nlu.train -c nlu_config.yml --fixed_model_name current \
--data data/intents/ -o /src_models --project nlu --verbose
/usr/local/lib/python3.6/runpy.py:125: RuntimeWarning: 'rasa_nlu.train' found in sys.modules after import of package 'rasa_nlu', but prior to execution of 'rasa_nlu.train'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)

2019-09-24 17:24:57.940222: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2019-09-24 17:24:57.969157: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2194975000 Hz
2019-09-24 17:24:57.969679: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x5631452c7380 executing computations on platform Host. Devices:
2019-09-24 17:24:57.969749: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-09-24 17:24:58 INFO     rasa_nlu.classifiers.embedding_intent_classifier  - Accuracy is updated every 10 epochs
Epochs: 100%|██████████| 300/300 [00:30<00:00,  9.83it/s, loss=0.070, acc=1.000]
2019-09-24 17:25:28 INFO     rasa_nlu.classifiers.embedding_intent_classifier  - Finished training embedding classifier, loss=0.070, train accuracy=1.000
2019-09-24 17:25:28 INFO     rasa_nlu.model  - Finished training compone

2019-09-24 17:25:37 WARNING  tensorflow  - From /usr/local/lib/python3.6/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.
2019-09-24 17:25:37 WARNING  tensorflow  - From /usr/local/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:102: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/25
562/562 [==============================] - 2s 3ms/sample - loss: 3.0762 - acc: 0.4295
Epoch 2/25
562/562 [==============================] - 1s 2ms/sample - loss: 2.3152 - acc: 0.4515
Epoch 3/25
562/562 [==============================] - 1s 2ms/sample - loss: 2.0175 - acc: 0.5175
Epoch 4/25
562/562 [==============================] - 1s 2ms/sample - loss: 1.7702 - acc: 0.6740
Epoch 5/25
562/562 [=============

* `rasa_core.evaluate` é o método utilizado para verificar a qualidade das suas `stories`.

In [6]:
!python -m rasa_core.evaluate --core $COACH_MODELS_DIALOGUE_PATH -s $COACH_STORIES_PATH -o results

Calling `rasa_core.evaluate` is deprecated. Please use `rasa_core.test` instead.
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/site-package

INFO:rasa_nlu.test:Confusion matrix, without normalization: 
[[91  0  0 ...  0  0  0]
 [ 0  2  0 ...  0  0  0]
 [ 0  0  1 ...  0  0  0]
 ...
 [ 0  0  0 ...  2  0  0]
 [ 0  0  0 ...  0  2  0]
 [ 0  0  0 ...  0  0  2]]
INFO:rasa_core.test:Finished evaluation


* Caso algum problme seja encotrado ele será descrito no arquivo `failed_stories.md`

In [7]:
!cat results/failed_stories.md

<!-- All stories passed -->

* Se tudo estiver correto você deverá ver a mensagem:

`<!-- All stories passed -->`

* Outro arquivo gerado é  o `story_confmat.pdf` uma matriz de confuzão onde é possível visualizar a relação entre as `utters` (mensagens enviadas .
`Para visualizar a matriz de confuzão neste notebook, abra-o com firefox`.

In [8]:
from IPython.display import IFrame
IFrame("results/story_confmat.pdf", width=900, height=900)

## Referências:

O Rasa está em constante evolução, alguns links úteis para a construção deste jupyter-notebook e para a análise das `stories` são:

* [Evaluation](https://rasa.com/docs/core/evaluation/)
* [Debugging](https://rasa.com/docs/core/debugging/)